In [ ]:
# importing the necessary libraries
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
np.random.seed(2)

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from importlib import reload
reload(plt)

import seaborn as sns
sns.set(style='white', context='notebook', palette='deep')

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import ReduceLROnPlateau

%matplotlib inline

In [ ]:
# Load the data
train = pd.read_csv("./input/train.csv")
test = pd.read_csv("./input/test.csv")

In [ ]:
X_train = train.drop(labels = ["label"],axis = 1)
y_train = train['label']

In [ ]:
# Feature Scaling
X_train = X_train/255

test = test/255

In [ ]:
y_train = to_categorical(y_train)

In [ ]:
X_train = X_train.values.reshape(X_train.shape[0],28,28,1)
test = test.values.reshape(test.shape[0],28,28,1)

In [ ]:
plt.imshow(X_train[0][:,:,0])

In [ ]:
random_seed = 2
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state=random_seed, stratify=y_train)

In [ ]:
#LeNet architecture

model = Sequential()

model.add( Conv2D( filters = 20, 
                   kernel_size = (5,5),
                   padding = 'same',
                   activation = 'relu',
                   input_shape = (28,28,1), data_format = 'channels_last' 
                 ) 
         )

model.add( MaxPool2D( pool_size = (2,2),
                      strides = (2,2)
                     ) 
          )


model.add( Conv2D( filters = 50, 
                   kernel_size = (5,5),
                   padding = 'same',
                   activation = 'relu', 
                 ) 
         )

model.add( MaxPool2D( pool_size = (2,2),
                      strides = (2,2)
                     ) 
          )

model.add(Flatten())

model.add(Dense(500, activation = 'relu'))

model.add(Dense(10, activation = 'softmax'))

In [ ]:
# Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [ ]:
# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
epochs = 2
batch_size = 86

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [ ]:
#Fit the model
history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size, callbacks=[learning_rate_reduction]
                              )

In [ ]:
#predict results
results = model.predict(test)

In [ ]:
results[0]

In [ ]:
# select the index with the maximum probability
results = np.argmax(results,axis = 1)


In [ ]:
results[0]

In [ ]:
results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("lenet_mnist_predictions.csv",index=False)

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model2.png', show_shapes=True, show_layer_names=True, rankdir='TB')

In [ ]:
model.save('lenet_kaggle.h5')

In [ ]:
import h5py
f = h5py.File('lenet_kaggle.h5', 'r')
f.keys()

In [ ]:
X_train.shape, X_val.shape, test.shape